### Import Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import torch
from torchvision import datasets, transforms, models  # datsets  , transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime

In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

### Import Dataset

<b> Dataset Link (Plant Vliiage Dataset ):</b><br> <a href='https://data.mendeley.com/datasets/tywbtsjrjv/1'> https://data.mendeley.com/datasets/tywbtsjrjv/1 </a> 

In [4]:
transform = transforms.Compose(
    [transforms.Resize(255), transforms.CenterCrop(224), transforms.ToTensor()]
)

<IPython.core.display.Javascript object>

In [5]:
dataset = datasets.ImageFolder("Dataset", transform=transform)

<IPython.core.display.Javascript object>

In [6]:
dataset

Dataset ImageFolder
    Number of datapoints: 55448
    Root location: Dataset
    StandardTransform
Transform: Compose(
               Resize(size=255, interpolation=bilinear, max_size=None, antialias=warn)
               CenterCrop(size=(224, 224))
               ToTensor()
           )

<IPython.core.display.Javascript object>

In [7]:
indices = list(range(len(dataset)))


<IPython.core.display.Javascript object>

In [8]:
split = int(np.floor(0.85 * len(dataset)))  # train_size

<IPython.core.display.Javascript object>

In [9]:
validation = int(np.floor(0.70 * split))  # validation

<IPython.core.display.Javascript object>

In [10]:
print(0, validation, split, len(dataset))

0 32991 47130 55448


<IPython.core.display.Javascript object>

In [11]:
print(f"length of train size :{validation}")
print(f"length of validation size :{split - validation}")
print(f"length of test size :{len(dataset)-validation}")

length of train size :32991
length of validation size :14139
length of test size :22457


<IPython.core.display.Javascript object>

In [12]:
np.random.shuffle(indices)

<IPython.core.display.Javascript object>

### Split into Train and Test

In [13]:
train_indices, validation_indices, test_indices = (
    indices[:validation],
    indices[validation:split],
    indices[split:],
)

<IPython.core.display.Javascript object>

In [14]:
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
test_sampler = SubsetRandomSampler(test_indices)

<IPython.core.display.Javascript object>

In [15]:
targets_size = len(dataset.class_to_idx)

<IPython.core.display.Javascript object>

### Model

<b>Convolution Aithmetic Equation : </b>(W - F + 2P) / S + 1 <br>
W = Input Size<br>
F = Filter Size<br>
P = Padding Size<br>
S = Stride <br>

### Transfer Learning

In [16]:
model = models.vgg16(pretrained=True)

C:\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<IPython.core.display.Javascript object>

In [17]:
for params in model.parameters():
    params.requires_grad = False

<IPython.core.display.Javascript object>

In [18]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

<IPython.core.display.Javascript object>

In [19]:
n_features = model.classifier[0].in_features
n_features

25088

<IPython.core.display.Javascript object>

In [20]:
model.classifier = nn.Sequential(
    nn.Linear(n_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(1024, targets_size),
)

<IPython.core.display.Javascript object>

In [21]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

<IPython.core.display.Javascript object>

### Original Modeling

In [16]:
class CNN(nn.Module):
    def __init__(self, K):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            # conv1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            # conv2
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            # conv3
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            # conv4
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
        )

        self.dense_layers = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(50176, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, K),
        )

    def forward(self, X):
        out = self.conv_layers(X)

        # Flatten
        out = out.view(-1, 50176)

        # Fully connected
        out = self.dense_layers(out)

        return out

<IPython.core.display.Javascript object>

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


<IPython.core.display.Javascript object>

In [18]:
device = "cpu"

<IPython.core.display.Javascript object>

In [19]:
model = CNN(targets_size)

<IPython.core.display.Javascript object>

In [20]:
model.to(device)

CNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

<IPython.core.display.Javascript object>

In [21]:
from torchsummary import summary

summary(model, (3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 256, 14, 14]         --
|    └─Conv2d: 2-1                       [-1, 32, 224, 224]        896
|    └─ReLU: 2-2                         [-1, 32, 224, 224]        --
|    └─BatchNorm2d: 2-3                  [-1, 32, 224, 224]        64
|    └─Conv2d: 2-4                       [-1, 32, 224, 224]        9,248
|    └─ReLU: 2-5                         [-1, 32, 224, 224]        --
|    └─BatchNorm2d: 2-6                  [-1, 32, 224, 224]        64
|    └─MaxPool2d: 2-7                    [-1, 32, 112, 112]        --
|    └─Conv2d: 2-8                       [-1, 64, 112, 112]        18,496
|    └─ReLU: 2-9                         [-1, 64, 112, 112]        --
|    └─BatchNorm2d: 2-10                 [-1, 64, 112, 112]        128
|    └─Conv2d: 2-11                      [-1, 64, 112, 112]        36,928
|    └─ReLU: 2-12                        [-1, 64, 112, 112]        --
| 

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 256, 14, 14]         --
|    └─Conv2d: 2-1                       [-1, 32, 224, 224]        896
|    └─ReLU: 2-2                         [-1, 32, 224, 224]        --
|    └─BatchNorm2d: 2-3                  [-1, 32, 224, 224]        64
|    └─Conv2d: 2-4                       [-1, 32, 224, 224]        9,248
|    └─ReLU: 2-5                         [-1, 32, 224, 224]        --
|    └─BatchNorm2d: 2-6                  [-1, 32, 224, 224]        64
|    └─MaxPool2d: 2-7                    [-1, 32, 112, 112]        --
|    └─Conv2d: 2-8                       [-1, 64, 112, 112]        18,496
|    └─ReLU: 2-9                         [-1, 64, 112, 112]        --
|    └─BatchNorm2d: 2-10                 [-1, 64, 112, 112]        128
|    └─Conv2d: 2-11                      [-1, 64, 112, 112]        36,928
|    └─ReLU: 2-12                        [-1, 64, 112, 112]        --
| 

<IPython.core.display.Javascript object>

In [22]:
criterion = nn.CrossEntropyLoss()  # this include softmax + cross entropy loss
optimizer = torch.optim.Adam(model.parameters())

<IPython.core.display.Javascript object>

### Batch Gradient Descent

In [23]:
def batch_gd(model, criterion, train_loader, test_loader, epochs):
    train_losses = np.zeros(epochs)
    validation_losses = np.zeros(epochs)

    for e in range(epochs):
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            output = model(inputs)

            loss = criterion(output, targets)

            train_loss.append(loss.item())  # torch to numpy world

            loss.backward()
            optimizer.step()

        train_loss = np.mean(train_loss)

        validation_loss = []

        for inputs, targets in test_loader:

            inputs, targets = inputs.to(device), targets.to(device)

            output = model(inputs)

            loss = criterion(output, targets)

            validation_loss.append(loss.item())  # torch to numpy world

        validation_loss = np.mean(validation_loss)

        train_losses[e] = train_loss
        validation_losses[e] = validation_loss

        dt = datetime.now() - t0

        print(
            f"Epoch : {e+1}/{epochs} Train_loss:{train_loss:.3f} Test_loss:{validation_loss:.3f} Duration:{dt}"
        )

    return train_losses, validation_losses

<IPython.core.display.Javascript object>

In [24]:
device = "cpu"

<IPython.core.display.Javascript object>

In [25]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=train_sampler
)
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=test_sampler
)
validation_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=validation_sampler
)

<IPython.core.display.Javascript object>

In [32]:
train_losses, validation_losses = batch_gd(
    model, criterion, train_loader, validation_loader, 1
)

Epoch : 1/1 Train_loss:2.227 Test_loss:1.356 Duration:4:56:09.250206


<IPython.core.display.Javascript object>

### Save the Model

In [38]:
torch.save(model.state_dict(), "plant_disease_model_1.pt")

<IPython.core.display.Javascript object>

### Load Model

In [26]:
targets_size = 39
model = CNN(targets_size)
model.load_state_dict(torch.load("plant_disease_model_1_latest.pt"))
model.eval()

CNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

<IPython.core.display.Javascript object>

In [41]:
# %matplotlib notebook

<IPython.core.display.Javascript object>

### Plot the loss

In [27]:
plt.plot(train_losses, label="train_loss")
plt.plot(validation_losses, label="validation_loss")
plt.xlabel("No of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

NameError: name 'train_losses' is not defined

<IPython.core.display.Javascript object>

### Accuracy

In [28]:
def accuracy(loader):
    n_correct = 0
    n_total = 0

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)

        _, predictions = torch.max(outputs, 1)

        n_correct += (predictions == targets).sum().item()
        n_total += targets.shape[0]

    acc = n_correct / n_total
    return acc

<IPython.core.display.Javascript object>

In [ ]:
train_acc = accuracy(train_loader)
test_acc = accuracy(test_loader)
validation_acc = accuracy(validation_loader)

In [97]:
print(
    f"Train Accuracy : {train_acc}\nTest Accuracy : {test_acc}\nValidation Accuracy : {validation_acc}"
)

Train Accuracy : 0.02003576733048407
Test Accuracy : 0.017071411396970427
Validation Accuracy : 0.017610863568852112


<IPython.core.display.Javascript object>

### Single Image Prediction

In [46]:
transform_index_to_disease = dataset.class_to_idx

<IPython.core.display.Javascript object>

In [47]:
transform_index_to_disease = dict(
    [(value, key) for key, value in transform_index_to_disease.items()]
)  # reverse the index

<IPython.core.display.Javascript object>

In [53]:
data = pd.read_csv("disease_info.csv", encoding="cp1252")

<IPython.core.display.Javascript object>

In [49]:
from PIL import Image
import torchvision.transforms.functional as TF

<IPython.core.display.Javascript object>

In [50]:
def single_prediction(image_path):
    image = Image.open(image_path)
    image = image.resize((224, 224))
    input_data = TF.to_tensor(image)
    input_data = input_data.view((-1, 3, 224, 224))
    output = model(input_data)
    output = output.detach().numpy()
    index = np.argmax(output)
    print("Original : ", image_path[12:-4])
    pred_csv = data["disease_name"][index]
    print(pred_csv)

<IPython.core.display.Javascript object>

In [94]:
single_prediction("test_images/Apple_ceder_apple_rust.JPG")

Original :  Apple_ceder_apple_rust
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

### Wrong Prediction

In [96]:
single_prediction("test_images/Apple_scab.JPG")

Original :  Apple_scab
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [56]:
single_prediction("test_images/Grape_esca.JPG")

Original :  Grape_esca
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [57]:
single_prediction("test_images/apple_black_rot.JPG")

Original :  apple_black_rot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [58]:
single_prediction("test_images/apple_healthy.JPG")

Original :  apple_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [59]:
single_prediction("test_images/background_without_leaves.jpg")

Original :  background_without_leaves
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [60]:
single_prediction("test_images/blueberry_healthy.JPG")

Original :  blueberry_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [61]:
single_prediction("test_images/cherry_healthy.JPG")

Original :  cherry_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [62]:
single_prediction("test_images/cherry_powdery_mildew.JPG")

Original :  cherry_powdery_mildew
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [63]:
single_prediction("test_images/corn_cercospora_leaf.JPG")

Original :  corn_cercospora_leaf
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [64]:
single_prediction("test_images/corn_common_rust.JPG")

Original :  corn_common_rust
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [65]:
single_prediction("test_images/corn_healthy.jpg")

Original :  corn_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [66]:
single_prediction("test_images/corn_northen_leaf_blight.JPG")

Original :  corn_northen_leaf_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [67]:
single_prediction("test_images/grape_black_rot.JPG")

Original :  grape_black_rot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [68]:
single_prediction("test_images/grape_healthy.JPG")

Original :  grape_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [69]:
single_prediction("test_images/grape_leaf_blight.JPG")

Original :  grape_leaf_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [70]:
single_prediction("test_images/orange_haunglongbing.JPG")

Original :  orange_haunglongbing
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [71]:
single_prediction("test_images/peach_bacterial_spot.JPG")

Original :  peach_bacterial_spot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [72]:
single_prediction("test_images/peach_healthy.JPG")

Original :  peach_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [73]:
single_prediction("test_images/pepper_bacterial_spot.JPG")

Original :  pepper_bacterial_spot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [74]:
single_prediction("test_images/pepper_bell_healthy.JPG")

Original :  pepper_bell_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [75]:
single_prediction("test_images/potato_early_blight.JPG")

Original :  potato_early_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [76]:
single_prediction("test_images/potato_healthy.JPG")

Original :  potato_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [77]:
single_prediction("test_images/potato_late_blight.JPG")

Original :  potato_late_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [78]:
single_prediction("test_images/raspberry_healthy.JPG")

Original :  raspberry_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [79]:
single_prediction("test_images/soyaben healthy.JPG")

Original :  soyaben healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [80]:
single_prediction("test_images/potato_late_blight.JPG")

Original :  potato_late_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [81]:
single_prediction("test_images/squash_powdery_mildew.JPG")

Original :  squash_powdery_mildew
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [82]:
single_prediction("test_images/starwberry_healthy.JPG")

Original :  starwberry_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [83]:
single_prediction("test_images/starwberry_leaf_scorch.JPG")

Original :  starwberry_leaf_scorch
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [84]:
single_prediction("test_images/tomato_bacterial_spot.JPG")

Original :  tomato_bacterial_spot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [85]:
single_prediction("test_images/tomato_early_blight.JPG")

Original :  tomato_early_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [86]:
single_prediction("test_images/tomato_healthy.JPG")

Original :  tomato_healthy
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [87]:
single_prediction("test_images/tomato_late_blight.JPG")

Original :  tomato_late_blight
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [88]:
single_prediction("test_images/tomato_leaf_mold.JPG")

Original :  tomato_leaf_mold
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [89]:
single_prediction("test_images/tomato_mosaic_virus.JPG")

Original :  tomato_mosaic_virus
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [90]:
single_prediction("test_images/tomato_septoria_leaf_spot.JPG")

Original :  tomato_septoria_leaf_spot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [91]:
single_prediction("test_images/tomato_spider_mites_two_spotted_spider_mites.JPG")

Original :  tomato_spider_mites_two_spotted_spider_mites
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [92]:
single_prediction("test_images/tomato_target_spot.JPG")

Original :  tomato_target_spot
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>

In [93]:
single_prediction("test_images/tomato_yellow_leaf_curl_virus.JPG")

Original :  tomato_yellow_leaf_curl_virus
Cherry : Powdery Mildew


<IPython.core.display.Javascript object>